In [3]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


## Read table
fname = 'vetting-v01'
fpath = f'../mnt/tess/labels/{fname}.csv'
all_table = pd.read_csv(fpath, header=0, low_memory=False).set_index('Astro ID')

## Rename or drop columns
all_table.drop(columns=['Split'])
all_table = all_table.rename(columns={'filename': 'File', 
                                      'Period': 'Per', 
                                      'Duration': 'Dur',
                                      'Transit_Depth': 'Depth',
                                      'star_rad': 'SRad',
                                      'star_rad_est': 'SRadEst',
                                      'star_mass': 'SMass',})

## Make label columns
disps = ['e', 'p', 'n', 'b', 't', 'u', 'j']
users = ['mk', 'ch', 'et', 'md', 'as', 'dm', 'Tansu', 'Shishir']
for d in disps:
    all_table[f'disp_{d}'] = 0

## Set labels
def set_labels(row):
    a = ~row.isna()
    if row['Final'] == 'i':
        # skip objects labeled as "inside the star"
        return row
    if a['Final']:
        row[f'disp_{row["Final"][0]}'] = 1
        row[f'disp_{row["Final"][1]}'] = 1
    else:
        for user in users:
            if a[user] and row[user]:
                row[f'disp_{row[user][0]}'] += 1
                row[f'disp_{row[user][1]}'] += 1

    return row

all_table = all_table.apply(set_labels, axis=1)

## Only use labelled rows 
print(f'Total entries: {len(all_table)}')
all_table = all_table[sum(all_table[f'disp_{d}'] for d in disps) > 0]
print(f'Total labeled entries: {len(all_table)}')
## And skip bad rows that have NaN in File', 'Per', 'Dur', 'Depth', or 'Tmag'
all_table = all_table[~all_table[['File', 'Per', 'Dur', 'Depth', 'Tmag']].isna().any(axis=1)]
print(f'Total after removing rows with missing File, Per, Dur, Depth, or Tmag: {len(all_table)}')


## Train-test split
t_train, t_test = train_test_split(all_table, test_size=0.1, random_state=42)
t_train, t_val = train_test_split(t_train, test_size=1./9, random_state=42)

## Print sizes of arrays and print duplicate counts
print(f'Split sizes. Train: {len(t_train)}; Valid: {len(t_val)}; Test: {len(t_test)}')
print(f'Duplicate TICs: {len(all_table.index.values) - len(set(all_table.index.values))}')
print('Splits')
print('  train:', len(t_train))
print('  val:', len(t_val))
print('  test:', len(t_test))

## Check label arrays
assert not any((t_train['disp_e'] + t_train['disp_p']+ t_train['disp_n'] + t_train['disp_b'] + t_train['disp_t'] + t_train['disp_u'] + t_train['disp_j']) == 0)
assert not any((t_val['disp_e'] + t_val['disp_p']+ t_val['disp_n'] + t_val['disp_b'] + t_val['disp_t'] + t_val['disp_u']+ t_val['disp_j']) == 0)
assert not any((t_test['disp_e'] + t_test['disp_p']+ t_test['disp_n'] + t_test['disp_b'] + t_test['disp_t'] + t_test['disp_u'] + t_test['disp_j']) == 0)

## Save train, test, and validation csv iles
t_train.to_csv(f'../mnt/tess/astronet/tces-{fname}-train.csv')
t_val.to_csv(f'../mnt/tess/astronet/tces-{fname}-val.csv')
t_test.to_csv(f'../mnt/tess/astronet/tces-{fname}-test.csv')
all_table.to_csv(f'../mnt/tess/astronet/tces-{fname}-all.csv')




Total entries: 3859
Total labeled entries: 3838
Total after removing rows with missing File, Per, Dur, Depth, or Tmag: 3778
Split sizes. Train: 3022; Valid: 378; Test: 378
Duplicate TICs: 0
Splits
  train: 3022
  val: 378
  test: 378


In [4]:
# all_table[['File', 'Per', 'Dur', 'Depth', 'Tmag', 'SRad', 'SRadEst', 'SMass']].isna().any(axis=1)
all_table

,TIC ID,Final,Decision,Distinct,mk,ch,et,md,as,dm,...,SRadEst,File,comment,disp_e,disp_p,disp_n,disp_b,disp_t,disp_u,disp_j
Astro ID,,,,,,,,,,,,,,,,,,,,,
1,101179364,et,NaN,1,et,et,et,et,et,NaN,...,1.098840,mk_hlsp_qlp_tess_ffi-s0013-0000000101179364_te...,E labels from triage,1,0,0,0,1,0,0
2,101255974,eb,eb,2,eb,eb,eb,eb,et,NaN,...,4.870666,mk_hlsp_qlp_tess_ffi-s0018-0000000101255974_te...,E labels from triage,1,0,0,1,0,0,0
3,101404344,eb,eb,2,eb,eb,eb,eb,pb,NaN,...,1.728549,mk_hlsp_qlp_tess_ffi-s0027-0000000101404344_te...,E labels from triage,1,0,0,1,0,0,0
4,101427335,et,et,2,et,et,et,eb,et,NaN,...,1.679462,mk_hlsp_qlp_tess_ffi-s0027-0000000101427335_te...,E labels from triage,1,0,0,0,1,0,0
5,10150705,eb,eb,2,eb,et,eb,eb,eb,NaN,...,0.794777,mk_hlsp_qlp_tess_ffi-s0022-0000000010150705_te...,E labels from triage,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3854,2003022879,NaN,NaN,2,eb,eb,NaN,NaN,NaN,NaN,...,2.311270,mk_hlsp_qlp_tess_ffi-s0016-0000002003022879_te...,E labels from triage,2,0,0,2,0,0,2
3855,2003049638,NaN,NaN,2,eb,eb,NaN,NaN,NaN,NaN,...,1.421851,mk_hlsp_qlp_tess_ffi-s0016-0000002003049638_te...,EBs from TOI group vetting,3,0,0,2,1,0,0
3856,2014876461,NaN,NaN,2,et,NaN,NaN,NaN,NaN,NaN,...,1.344808,mk_hlsp_qlp_tess_ffi-s0016-0000002014876461_te...,EBs from TOI group vetting,2,0,0,1,1,0,0
